## Linkage of openTEPES to the openENTRANCE Platform

In [1]:
import pyam

# set user name to get access to the scenario explorer
user_name = ''
user_password = ''

pyam.iiasa.set_config(user_name, user_password)

<IPython.core.display.Javascript object>

pyam - INFO: Running in a notebook, setting `pyam` logging level to `logging.INFO` and adding stderr handler
pyam.iiasa - INFO: Setting IIASA-connection configuration file: C:\Users\Erik AQ\.local\pyam\iiasa.yaml


## Tool 1: Reading data from openENTRANCE Platform

### Definition: Model, scenario and region name

In [2]:
import pyam 

model_name = 'GENeSYS-MOD 2.9.0-oe'
scenario_name = 'Societal Commitment 1.0'
region_name = 'Spain'

### Connection to the platform and saving variables as pandas dataframe

In [3]:
pyam.iiasa.Connection('openentrance')
# Reading data from openENTRANCE Platform
df = pyam.read_iiasa('openentrance', model = model_name, scenario = scenario_name)
#Power generation capacities for Spain
gen_cap = df.filter(region = 'Spain',variable = 'Capacity|Electricity|*', year = 2030).as_pandas()
gen_cap.head()

pyam.iiasa - INFO: You are connected to the IXSE_OPENENTRANCE scenario explorer hosted by IIASA. If you use this data in any published format, please cite the data as provided in the explorer guidelines: https://data.ene.iiasa.ac.at/openentrance/#/about
pyam.iiasa - INFO: You are connected as user `erikfilias`
pyam.iiasa - INFO: You are connected to the IXSE_OPENENTRANCE scenario explorer hosted by IIASA. If you use this data in any published format, please cite the data as provided in the explorer guidelines: https://data.ene.iiasa.ac.at/openentrance/#/about
pyam.iiasa - INFO: You are connected as user `erikfilias`


,model,scenario,region,variable,unit,year,value,exclude,version
0,GENeSYS-MOD 2.9.0-oe,Societal Commitment 1.0,Spain,Capacity|Electricity|Biomass,GW,2030,0.849037,False,1
1,GENeSYS-MOD 2.9.0-oe,Societal Commitment 1.0,Spain,Capacity|Electricity|Coal,GW,2030,4.266979,False,1
2,GENeSYS-MOD 2.9.0-oe,Societal Commitment 1.0,Spain,Capacity|Electricity|Gas,GW,2030,31.219368,False,1
3,GENeSYS-MOD 2.9.0-oe,Societal Commitment 1.0,Spain,Capacity|Electricity|Hydro,GW,2030,32.492324,False,1
4,GENeSYS-MOD 2.9.0-oe,Societal Commitment 1.0,Spain,Capacity|Electricity|Nuclear,GW,2030,7.582040,False,1


## Tool 1: Reading data from openENTRANCE Platform

In [4]:
pyam.iiasa.Connection('openentrance')
# Reading data from openENTRANCE Platform
df = pyam.read_iiasa('openentrance', model = 'openTEPES 1.6.32', scenario = 'Baseline|SEP2030oE')
#Power transmission capacities for Spain
line_cap = df.as_pandas()
line_cap.head()

pyam.iiasa - INFO: You are connected to the IXSE_OPENENTRANCE scenario explorer hosted by IIASA. If you use this data in any published format, please cite the data as provided in the explorer guidelines: https://data.ene.iiasa.ac.at/openentrance/#/about
pyam.iiasa - INFO: You are connected as user `erikfilias`
pyam.iiasa - INFO: You are connected to the IXSE_OPENENTRANCE scenario explorer hosted by IIASA. If you use this data in any published format, please cite the data as provided in the explorer guidelines: https://data.ene.iiasa.ac.at/openentrance/#/about
pyam.iiasa - INFO: You are connected as user `erikfilias`


,model,scenario,region,variable,unit,year,subannual,value,exclude,version
0,openTEPES 1.6.32,Baseline|SEP2030oE,ES11>ES12,Maximum Flow|Electricity|Grid,MW,2030,January,50000.0,False,2
1,openTEPES 1.6.32,Baseline|SEP2030oE,ES11>ES41,Maximum Flow|Electricity|Grid,MW,2030,January,50000.0,False,2
2,openTEPES 1.6.32,Baseline|SEP2030oE,ES12>ES13,Maximum Flow|Electricity|Grid,MW,2030,January,50000.0,False,2
3,openTEPES 1.6.32,Baseline|SEP2030oE,ES12>ES41,Maximum Flow|Electricity|Grid,MW,2030,January,50000.0,False,2
4,openTEPES 1.6.32,Baseline|SEP2030oE,ES13>ES21,Maximum Flow|Electricity|Grid,MW,2030,January,50000.0,False,2


## Tool 2: Data tranformation to openTEPES format

In [5]:
import pandas as pd

model_name    = line_cap['model'].unique()
scenario_name = gen_cap['scenario'].unique()

gen_variable  = gen_cap['variable'].unique()
gen_variable  = pd.DataFrame(gen_variable)
gen_variable  = gen_variable[0].str.split('|', expand = True)
gen_dict      = gen_variable[2]

line_variable = line_cap['variable'].unique()
line_region   = line_cap['region']
line_region   = pd.DataFrame(line_region)
line_region   = line_region['region'].str.split('>', expand = True)

### Power Generation Data to openTEPES format

In [6]:
import numpy
a = numpy.zeros(shape=(len(gen_dict.values),28))
dfGeneration = pd.DataFrame(a,columns=['Node','Technology','StorageType','MustRun','MaximumPower','MinimumPower'
                                       ,'MaximumCharge','InitialStorage','MaximumStorage','MinimumStorage'
                                       ,'Efficiency','EFOR','RampUp','RampDown'
                                       ,'UpTime','DownTime','FuelCost','LinearTerm','ConstantTerm'
                                       ,'OMVariableCost','StartUpCost','ShutDownCost','CO2EmissionRate','FixedCost'
                                       ,'FixedChargeRate','BinaryInvestment','MaximumReactivePower'
                                       ,'MinimumReactivePower'])
dfGeneration['Node']         = gen_cap['region']
dfGeneration['Technology']   = gen_dict.values
dfGeneration['MaximumPower'] = gen_cap['value']*1e3
dfGeneration = dfGeneration.set_index(gen_dict.values)
dfGeneration.to_csv('oT_Data_Generation_TEST.csv', sep=',')

### Transmission Network Data to openTEPES format

In [7]:
import numpy
a = numpy.zeros(shape=(len(line_region),16))
dfNetwork = pd.DataFrame(a,columns=['From','To','Circuit','LineType','Voltage','LossFactor','Reactance','TTC'
                                    ,'SecurityFactor','FixedCost','FixedChargeRate','BinaryInvestment'
                                    ,'Resistance','Susceptance','Tap','Converter'])
dfNetwork['From'   ]         = line_region[0]
dfNetwork['To'     ]         = line_region[1]
dfNetwork['Circuit']         = 'cc1'
dfNetwork['TTC'    ]         = line_cap['value']
dfNetwork                    = dfNetwork.set_index(['From', 'To', 'Circuit'])
dfNetwork                    = dfNetwork.rename_axis([None, None, None], axis=0)
dfNetwork.to_csv('oT_Data_Network_TEST.csv', sep=',')

## Tool 3: Write to IAMC format

In [8]:
model_name    = 'openTEPES 1.6.32'
scenario_name = 'Baseline|SEP2030oE'
variable_name = 'Maximum Flow|Electricity|Grid'
unit_name     = 'MW'
year_name     = 2030
subannual_name= 'January'

In [9]:
dfNetworkExp  = pd.read_csv('oT_Data_Network_TEST.csv')

In [10]:
region_name   = dfNetworkExp['Unnamed: 0']+'>'+dfNetworkExp['Unnamed: 1']

In [11]:
def write_to_IAMCformat(model, scenario, region, variable, unit, year, subannual, value):
    _df = pd.DataFrame({'model': model,
                       'scenario': scenario,
                       'region': region,
                       'variable': variable,
                       'unit': unit,
                       'year': year,
                       'subannual': subannual,
                       'value': value,})
    return _df

In [12]:
dfNetwork_IAMC = write_to_IAMCformat(model_name, scenario_name, 
                                     region_name, variable_name, unit_name, year_name, subannual_name, dfNetworkExp['TTC'])

In [13]:
dfNetwork_IAMC.head()

,model,scenario,region,variable,unit,year,subannual,value
0,openTEPES 1.6.32,Baseline|SEP2030oE,ES11>ES12,Maximum Flow|Electricity|Grid,MW,2030,January,50000.0
1,openTEPES 1.6.32,Baseline|SEP2030oE,ES11>ES41,Maximum Flow|Electricity|Grid,MW,2030,January,50000.0
2,openTEPES 1.6.32,Baseline|SEP2030oE,ES12>ES13,Maximum Flow|Electricity|Grid,MW,2030,January,50000.0
3,openTEPES 1.6.32,Baseline|SEP2030oE,ES12>ES41,Maximum Flow|Electricity|Grid,MW,2030,January,50000.0
4,openTEPES 1.6.32,Baseline|SEP2030oE,ES13>ES21,Maximum Flow|Electricity|Grid,MW,2030,January,50000.0


## Tool 4: Validating the output files using the validate function from nomenclature

In [14]:
import nomenclature as nc
c = nc.validate(dfNetwork_IAMC)
print(c)

True
